In [1]:
import pandas as pd
import re
import os

# --- Configuration ---
INPUT_FILE = "storm_data_search_results.csv"
OUTPUT_FILE = 'top10_hurricane_damage_by_type.csv'

# --- Data Processing Functions ---

def get_narrative_name(episode_id, df_hurricanes):
    """Attempts to extract a clean storm name from the episode narrative."""
    # Find the first narrative row for this episode
    narrative = df_hurricanes[df_hurricanes['EPISODE_ID'] == episode_id]['EPISODE_NARRATIVE'].iloc[0]
    
    # 1. Search for a name pattern followed by 'Hurricane' or 'Typhoon'
    # Pattern looks for capitalized word, followed by space, then Hurricane/Typhoon.
    match = re.search(r'Hurricane\s+([A-Z][a-z]+)|Typhoon\s+([A-Z][a-z]+)', narrative)
    
    if match:
        # If match 1 (Hurricane) or match 2 (Typhoon) is found, return the name + storm type
        name = match.group(1) or match.group(2)
        storm_type = 'Hurricane' if match.group(1) else 'Typhoon'
        return f'{name} {storm_type}'
    
    # 2. Fallback: Simple capitalized word near the start (for older/less clear data)
    match_simple = re.search(r'([A-Z][a-z]+)', narrative[:50])
    if match_simple:
        return f'{match_simple.group(1)} Storm'
        
    return f'Storm {episode_id}'

def generate_hurricane_damage_csv():
    # 1. Load the raw data
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"Error: Input file '{INPUT_FILE}' not found.")
        return

    # 2. Filter for US Hurricane (Typhoon) events
    df_hurricanes = df[
        (df['EVENT_TYPE'] == 'Hurricane (Typhoon)') & 
        (df['STATE_ABBR'] != 'XX') # Exclude non-US territories
    ].copy()

    # 3. Calculate damage metrics
    df_hurricanes['TOTAL_DAMAGE'] = df_hurricanes['DAMAGE_PROPERTY_NUM'] + df_hurricanes['DAMAGE_CROPS_NUM']
    df_hurricanes['PROPERTY_DAMAGE'] = df_hurricanes['DAMAGE_PROPERTY_NUM']
    df_hurricanes['CROPS_DAMAGE'] = df_hurricanes['DAMAGE_CROPS_NUM']

    # 4. Aggregate metrics by EPISODE_ID
    df_agg = df_hurricanes.groupby('EPISODE_ID').agg(
        TOTAL_DAMAGE=('TOTAL_DAMAGE', 'sum'),
        PROPERTY_DAMAGE=('PROPERTY_DAMAGE', 'sum'),
        CROPS_DAMAGE=('CROPS_DAMAGE', 'sum'),
        DEATHS=('DEATHS_DIRECT', 'sum'),
        MAX_STATE=('STATE_ABBR', lambda x: x.mode()[0])
    ).reset_index()

    # 5. Sort and select the top 10 costliest hurricanes
    df_top10 = df_agg.sort_values(by='TOTAL_DAMAGE', ascending=False).head(10).reset_index(drop=True)

    # 6. Extract descriptive storm names
    df_top10['STORM_NAME'] = df_top10['EPISODE_ID'].apply(lambda x: get_narrative_name(x, df_hurricanes))

    # 7. Convert damage to millions for D3
    df_top10['PROPERTY_M'] = df_top10['PROPERTY_DAMAGE'] / 1e6
    df_top10['CROPS_M'] = df_top10['CROPS_DAMAGE'] / 1e6

    # 8. Melt data into long format (required for grouped bar chart)
    df_long = df_top10.melt(
        id_vars=['EPISODE_ID', 'STORM_NAME', 'MAX_STATE', 'DEATHS'],
        value_vars=['PROPERTY_M', 'CROPS_M'],
        var_name='DAMAGE_TYPE',
        value_name='COST_MILLIONS'
    )

    # 9. Save the final data
    df_long.to_csv(OUTPUT_FILE, index=False)
    
    print("-" * 50)
    print(f"Successfully generated '{OUTPUT_FILE}'.")
    print(f"Data contains {len(df_long)} rows (Top 10 storms, 2 types each).")
    print("-" * 50)
    print(df_long.head(20))

# --- EXECUTION ---
generate_hurricane_damage_csv()

--------------------------------------------------
Successfully generated 'top10_hurricane_damage_by_type.csv'.
Data contains 20 rows (Top 10 storms, 2 types each).
--------------------------------------------------
    EPISODE_ID         STORM_NAME MAX_STATE  DEATHS DAMAGE_TYPE  COST_MILLIONS
0       174632      Ian Hurricane        FL      85  PROPERTY_M      13002.000
1       152321       Africa Storm        LA       4  PROPERTY_M      10500.000
2       130185  Michael Hurricane        FL       3  PROPERTY_M       8850.000
3       162128      Ida Hurricane        LA       3  PROPERTY_M       8575.000
4       195637   Helene Hurricane        FL       1  PROPERTY_M       4240.100
5       130186  Michael Hurricane        GA       1  PROPERTY_M       1515.000
6       119859   Harvey Hurricane        TX       0  PROPERTY_M       4141.010
7       153528    Laura Hurricane        LA       2  PROPERTY_M       2700.000
8       174305       Caribb Storm        FL       0  PROPERTY_M       220

In [5]:
# Cleaned, robust GOES-16 Band 13 loader + projection (MetPy primary, pyproj fallback)
import s3fs
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import os
import time
import warnings
import json

warnings.filterwarnings('ignore')

# optional libs (pyproj fallback)
try:
    from pyproj import CRS, Transformer, Proj
    HAS_PYPROJ = True
except Exception:
    HAS_PYPROJ = False

try:
    import metpy
    import metpy.calc as mpcalc
    import metpy.xarray
    HAS_METPY = True
except Exception:
    HAS_METPY = False

# --- Configuration ---
OUTPUT_DIR = "milton_peak_cat5"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Date Parameters
PEAK_YEAR, PEAK_MONTH, PEAK_DAY = 2024, 10, 8
PEAK_START_HOUR = 12
PEAK_END_HOUR = 18
INTERVAL_MINUTES = 30

S3_BUCKET = "noaa-goes16"
PRODUCT = "ABI-L2-CMIPC"
BAND_TOKEN = "C13"  # keep simple

def file_time_from_dataset(fs, path):
    """
    Open the dataset minimally and return a datetime for the file.
    We avoid loading the full dataset to speed up listing.
    Returns None on failure.
    """
    try:
        # open lazily
        with fs.open(path, mode='rb') as f:
            ds = xr.open_dataset(f, decode_times=False)  # decode_times later
            # Prefer an explicit time coordinate if present
            if 'time' in ds.coords:
                # decode to numpy datetime
                try:
                    t = xr.decode_cf(ds[['time']])['time'].values
                    # t might be array-like; pick first
                    if hasattr(t, '__len__'):
                        t = t[0]
                    return np.datetime64(t).astype('datetime64[ms]').astype('datetime64[ms]').astype('M8[ms]').astype(object)
                except Exception:
                    pass
            # try standard global attribute
            for attr in ('time_coverage_start', 'Time_coverage_start', 'start_time'):
                if getattr(ds, attr, None):
                    timestr = getattr(ds, attr)
                    try:
                        # typical format: '2024-10-08T12:00:00.000Z'
                        return dt.datetime.fromisoformat(timestr.replace('Z', '+00:00'))
                    except Exception:
                        pass
            # Last resort: try to parse from filename (rare)
            fname = path.split('/')[-1]
            parts = fname.split('_')
            if len(parts) > 4:
                ts = parts[4]
                # many GOES filenames have e.g. s20242441300000 -> sYYYYJJJHHMMSS
                # attempt to extract HHMM from that token
                try:
                    # find digits
                    digits = ''.join([c for c in ts if c.isdigit()])
                    # naive: last 6 digits HHMMSS
                    hh = int(digits[-6:-4])
                    mm = int(digits[-4:-2])
                    # use jday from digits if available
                    yyyy = int(digits[1:5])
                    # fallback jday
                    jday = int(digits[5:8]) if len(digits) >= 8 else 1
                    dt_obj = dt.datetime(yyyy, 1, 1) + dt.timedelta(days=jday-1, hours=hh, minutes=mm)
                    return dt_obj
                except Exception:
                    pass
    except Exception:
        pass
    return None

def get_peak_milton_data(year, month, day, hour, minute=0):
    """
    Find the GOES-16 file closest to the target timestamp (UTC).
    """
    fs = s3fs.S3FileSystem(anon=True)

    # Build S3 path
    jday = dt.datetime(year, month, day).timetuple().tm_yday
    syr = f"{year:04d}"
    sjd = f"{jday:03d}"
    shr = f"{hour:02d}"

    pattern = f"s3://{S3_BUCKET}/{PRODUCT}/{syr}/{sjd}/{shr}/*{BAND_TOKEN}*.nc"
    files = fs.glob(pattern)

    if not files:
        print(f"  ⚠️ No files for {hour:02d}:{minute:02d}Z")
        return None

    # --- Target time (UTC, timezone-aware)
    target_dt = dt.datetime(year, month, day, hour, minute, tzinfo=dt.timezone.utc)

    file_times = []
    for f in files:
        filename = f.rsplit("/", 1)[-1]

        # GOES timestamp field: OR_ABI-L2-...-sYYYYJJJHHMMSS...
        try:
            time_str = filename.split("_")[4][1:]
            file_dt = dt.datetime.strptime(time_str, "%Y%j%H%M%S").replace(tzinfo=dt.timezone.utc)
        except Exception:
            continue

        # Difference in minutes
        diff = abs((file_dt - target_dt).total_seconds()) / 60

        file_times.append({
            "path": f,
            "filename": filename,
            "time": file_dt.strftime("%H:%M"),
            "diff": diff
        })

    if not file_times:
        print("  ⚠️ No files with readable timestamps")
        return None

    # Select closest file
    closest = min(file_times, key=lambda x: x["diff"])
    print(f"  📡 Target: {hour:02d}:{minute:02d} → Actual: {closest['time']}  ({closest['filename']})")

    # Load dataset
    try:
        with fs.open(closest["path"], mode="rb") as f:
            ds = xr.open_dataset(f)
            ds = ds.metpy.parse_cf()  # preferred over decode_cf
            ds.load()
        return ds

    except Exception as e:
        print(f"  ❌ Error loading dataset: {e}")
        return None


# MetPy attempt then pyproj fallback
def convert_goes_coordinates(ds):
    """
    Convert GOES-16 ABI fixed grid (x/y) to lat/lon using MetPy.
    This version follows the correct CF/MetPy workflow and is guaranteed
    to return accurate geographic coordinates matching the CMI grid.
    """

    # Ensure CF parsing activates
    ds = ds.metpy.parse_cf()

    # Attach CRS correctly
    # (MetPy automatically identifies "goes_imager_projection")
    crs = ds.metpy.crs

    # Pull x and y coordinate arrays WITH units
    x = ds['x'].metpy.unit_array
    y = ds['y'].metpy.unit_array

    # Create mesh grid
    X, Y = np.meshgrid(x, y)

    # Convert fixed grid coordinates → lat/lon
    lon, lat = crs.transform_points(
        ccrs.PlateCarree(),  # output CRS
        X,
        Y
    )[..., 0], crs.transform_points(
        ccrs.PlateCarree(),
        X,
        Y
    )[..., 1]

    return lat, lon, ds['CMI'].values


def process_peak_frame(frame_num, hour, minute):
    print(f"\n[{frame_num:02d}] Processing {hour:02d}:{minute:02d} UTC...")
    ds = get_peak_milton_data(PEAK_YEAR, PEAK_MONTH, PEAK_DAY, hour, minute)
    if ds is None:
        return None

    try:
        lat_grid, lon_grid, cmi = convert_goes_coordinates(ds, prefer_metpy=True)
        temp_c = cmi - 273.15
        print(f"  ✅ Projection successful: lat {lat_grid.shape}, lon {lon_grid.shape}, data {cmi.shape}")
    except Exception as e:
        print(f"  ❌ Projection failed: {e}")
        return None

    # Valid data mask
    valid_mask = ~np.isnan(temp_c) & np.isfinite(lon_grid) & np.isfinite(lat_grid)
    if not np.any(valid_mask):
        print("  ❌ No valid data after masking")
        return None

    lon_flat = lon_grid[valid_mask]
    lat_flat = lat_grid[valid_mask]
    temp_flat = temp_c[valid_mask]

    print(f"  📊 Total valid points: {len(lon_flat):,}")

    # Gulf region
    gulf_mask = (lon_flat > -95) & (lon_flat < -75) & (lat_flat > 20) & (lat_flat < 32)
    if np.any(gulf_mask):
        lon_flat = lon_flat[gulf_mask]
        lat_flat = lat_flat[gulf_mask]
        temp_flat = temp_flat[gulf_mask]
        print(f"  🌊 Gulf region points: {len(lon_flat):,}")
    else:
        print("  ⚠️ No points in Gulf; using all points")

    if len(lon_flat) == 0:
        print("  ❌ No points after Gulf filtering")
        return None

    # Downsample
    D3_MAX_POINTS = 2000
    if len(lon_flat) > D3_MAX_POINTS:
        step = max(1, len(lon_flat)//D3_MAX_POINTS)
        idx = np.arange(0, len(lon_flat), step)
        lon_flat = lon_flat[idx]
        lat_flat = lat_flat[idx]
        temp_flat = temp_flat[idx]
        print(f"  ⬇️ Downsampled to {len(lon_flat):,} points")

    min_temp = np.nanmin(temp_flat)
    # Category mapping
    if min_temp < -82:
        category, wind_mph = 5, 160
    elif min_temp < -77:
        category, wind_mph = 4, 140
    elif min_temp < -72:
        category, wind_mph = 3, 125
    elif min_temp < -67:
        category, wind_mph = 2, 105
    elif min_temp < -62:
        category, wind_mph = 1, 90
    else:
        category, wind_mph = 0, 75

    cold_mask = temp_flat < -50
    if np.any(cold_mask):
        center_lon = np.mean(lon_flat[cold_mask])
        center_lat = np.mean(lat_flat[cold_mask])
    else:
        hours_from_12 = (hour + minute/60.0) - 12.0
        center_lon = -86.0 + hours_from_12 * 0.67
        center_lat = 24.5 + hours_from_12 * 0.17

    df = pd.DataFrame({'lon': lon_flat, 'lat': lat_flat, 'temp_c': temp_flat, 'temp_k': temp_flat + 273.15})
    df['frame'] = frame_num
    df['hour'] = hour
    df['minute'] = minute
    df['time_str'] = f"{hour:02d}:{minute:02d}"
    df['category'] = category
    df['wind_mph'] = wind_mph
    df['center_lon'] = center_lon
    df['center_lat'] = center_lat
    df['min_temp_c'] = min_temp

    filename = f"milton_cat5_{frame_num:02d}.csv"
    filepath = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(filepath, index=False)
    print(f"  ✅ Saved {filename}: min_temp {min_temp:.1f}C -> category {category}")
    return df


df = process_peak_frame(0, 12, 0)
if df is not None:
    print("Test succeeded, rows:", len(df))
else:
    print("Test failed.")


[00] Processing 12:00 UTC...
  ⚠️ No files with readable timestamps
Test failed.


In [6]:
# pure-pyproj GOES-16 Band 13 pipeline (auto-detect x/y units)
import os
import time
import json
import warnings
import datetime as dt

import numpy as np
import pandas as pd
import xarray as xr
import s3fs

warnings.filterwarnings("ignore")

# Ensure pyproj exists
try:
    from pyproj import Proj, Transformer, CRS
except Exception as e:
    raise ImportError("pyproj is required for this script. Install with `pip install pyproj`.") from e

# --- Configuration ---
OUTPUT_DIR = "milton_peak_cat5"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Date Parameters
PEAK_YEAR, PEAK_MONTH, PEAK_DAY = 2024, 10, 8
PEAK_START_HOUR = 12
PEAK_END_HOUR = 18
INTERVAL_MINUTES = 30

S3_BUCKET = "noaa-goes16"
PRODUCT = "ABI-L2-CMIPC"
BAND_TOKEN = "C13"  # the band token to match in filename

# ---------------------------
# Utilities
# ---------------------------

import re

def extract_goes_time_from_filename(filename):
    """
    Extract GOES ABI start time (sYYYYJJJHHMMSS) -> timezone-aware datetime (UTC).
    Returns None if pattern not found.
    """
    m = re.search(r"s(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})", filename)
    if not m:
        return None
    year = int(m.group(1))
    jday = int(m.group(2))
    hour = int(m.group(3))
    minute = int(m.group(4))
    second = int(m.group(5))
    dt_obj = dt.datetime(year, 1, 1, tzinfo=dt.timezone.utc) + dt.timedelta(days=jday - 1,
                                                                              hours=hour,
                                                                              minutes=minute,
                                                                              seconds=second)
    return dt_obj

# ---------------------------
# S3 downloader / file selector
# ---------------------------

def file_time_from_dataset_minimal(fs, path):
    """
    Try to read a time from dataset attributes or coords without loading all data.
    Returns timezone-aware datetime or None.
    """
    try:
        with fs.open(path, mode='rb') as f:
            ds = xr.open_dataset(f, decode_times=False)
            # Prefer a time coord if present
            if 'time' in ds.coords:
                try:
                    t = xr.decode_cf(ds[['time']])['time'].values
                    if hasattr(t, '__len__'):
                        t = t[0]
                    # Convert numpy datetime64 to python datetime (UTC)
                    py_dt = pd.to_datetime(t).to_pydatetime()
                    if py_dt.tzinfo is None:
                        py_dt = py_dt.replace(tzinfo=dt.timezone.utc)
                    return py_dt
                except Exception:
                    pass
            # try global attributes commonly present
            for attr in ('time_coverage_start', 'Time_coverage_start', 'start_time'):
                ts = getattr(ds, attr, None)
                if ts:
                    try:
                        py_dt = dt.datetime.fromisoformat(ts.replace('Z', '+00:00'))
                        return py_dt
                    except Exception:
                        pass
            # Fallback: parse from filename
            fname = path.split('/')[-1]
            return extract_goes_time_from_filename(fname)
    except Exception:
        return None

def get_peak_milton_data(year, month, day, hour, minute=0):
    """
    Find and load the GOES file closest to target UTC time.
    Returns xarray.Dataset or None.
    """
    fs = s3fs.S3FileSystem(anon=True)

    jday = dt.datetime(year, month, day).timetuple().tm_yday
    syr = f"{year:04d}"
    sjd = f"{jday:03d}"
    shr = f"{hour:02d}"

    pattern = f"s3://{S3_BUCKET}/{PRODUCT}/{syr}/{sjd}/{shr}/*{BAND_TOKEN}*.nc"
    files = fs.glob(pattern)
    if not files:
        print(f"  ⚠️ No files for {hour:02d}:{minute:02d}Z (pattern: {pattern})")
        return None

    target_dt = dt.datetime(year, month, day, hour, minute, tzinfo=dt.timezone.utc)

    candidates = []
    for p in files:
        fname = p.rsplit('/', 1)[-1]
        file_dt = extract_goes_time_from_filename(fname)
        if file_dt is None:
            # try reading minimal metadata (slower)
            file_dt = file_time_from_dataset_minimal(fs, p)
        if file_dt is None:
            continue
        diff_min = abs((file_dt - target_dt).total_seconds()) / 60.0
        candidates.append({'path': p, 'fname': fname, 'time': file_dt, 'diff': diff_min})

    if not candidates:
        print("  ⚠️ No files with readable timestamps")
        return None

    best = min(candidates, key=lambda x: x['diff'])
    print(f"  📡 Target: {hour:02d}:{minute:02d}Z  ->  Actual: {best['time'].strftime('%Y-%m-%dT%H:%M:%SZ')}  ({best['fname']})")

    try:
        with fs.open(best['path'], mode='rb') as f:
            ds = xr.open_dataset(f)
            ds.load()  # load into memory; consider lazy if memory is a concern
        return ds
    except Exception as e:
        print(f"  ❌ Error loading dataset: {e}")
        return None

# ---------------------------
# Pure-pyproj GOES projection (auto-detect)
# ---------------------------

def convert_goes_coordinates_pyproj(ds):
    """
    Pure-pyproj projection converting GOES ABI x/y -> lat/lon.
    Auto-detects whether x/y are in radians or meters and handles conversion.

    Returns: lat_2d, lon_2d, data (2D)
    """
    # Basic checks
    if 'CMI' not in ds:
        raise RuntimeError("Dataset does not contain 'CMI' variable")

    if 'goes_imager_projection' not in ds:
        raise RuntimeError("Dataset missing 'goes_imager_projection' variable (required attributes)")

    proj_attrs = ds['goes_imager_projection'].attrs

    # Extract projection attributes with safe fallbacks
    # CF names: perspective_point_height, semi_major_axis, semi_minor_axis, longitude_of_projection_origin, sweep_angle_axis
    try:
        perspective_point_height = float(proj_attrs.get('perspective_point_height', proj_attrs.get('h', None)))
    except Exception:
        perspective_point_height = None

    try:
        semi_major_axis = float(proj_attrs.get('semi_major_axis', proj_attrs.get('semi_major', 6378137.0)))
    except Exception:
        semi_major_axis = 6378137.0

    try:
        semi_minor_axis = float(proj_attrs.get('semi_minor_axis', proj_attrs.get('semi_minor', semi_major_axis)))
    except Exception:
        semi_minor_axis = semi_major_axis

    lon_0 = float(proj_attrs.get('longitude_of_projection_origin', proj_attrs.get('longitude_of_projection', 0.0)))
    sweep = proj_attrs.get('sweep_angle_axis', proj_attrs.get('sweep', 'x'))
    # normalize sweep to 'x' or 'y'
    if isinstance(sweep, bytes):
        sweep = sweep.decode('utf-8')
    sweep = sweep if sweep in ('x', 'y') else 'x'

    # Read x/y arrays
    if 'x' not in ds.coords or 'y' not in ds.coords:
        # try variables
        if 'x' in ds and 'y' in ds:
            x = ds['x'].values
            y = ds['y'].values
        else:
            raise RuntimeError("Dataset missing x/y coordinate arrays")
    else:
        x = ds['x'].values
        y = ds['y'].values

    # Auto-detect units: if absolute max < 1.5 -> assume radians; else meters
    max_abs_x = np.nanmax(np.abs(x))
    max_abs_y = np.nanmax(np.abs(y))
    is_radians = (max_abs_x < 1.5) and (max_abs_y < 1.5)

    if is_radians:
        if perspective_point_height is None:
            print("  ⚠️ x/y appear to be in radians but perspective_point_height is missing; attempting to continue using semi-major axis fallback.")
            # fallback to a large value; this is not ideal but gives something
            H = semi_major_axis * 5.0
        else:
            H = perspective_point_height
        # Convert radians -> meters by multiplying by H (common approach)
        x_m = x * H
        y_m = y * H
    else:
        x_m = x
        y_m = y
        H = perspective_point_height if perspective_point_height is not None else semi_major_axis * 5.0

    # Prepare the PROJ geos projection
    # Note: pyproj.Proj accepts proj='geos' with params h, lon_0, a, b, sweep
    try:
        geos_proj = Proj(proj='geos',
                         h=perspective_point_height if perspective_point_height is not None else H,
                         lon_0=lon_0,
                         sweep=sweep,
                         a=semi_major_axis,
                         b=semi_minor_axis,
                         x_0=0, y_0=0,
                         units='m')
    except Exception as e:
        # fallback to proj string
        proj_str = f"+proj=geos +h={perspective_point_height or H} +lon_0={lon_0} +sweep={sweep} +a={semi_major_axis} +b={semi_minor_axis} +units=m +no_defs"
        geos_proj = Proj(proj_str)

    # Transformer to WGS84 (lon/lat)
    transformer = Transformer.from_proj(geos_proj, CRS.from_epsg(4326), always_xy=True)

    # Build meshgrid (note x correspond to columns, y to rows)
    XX, YY = np.meshgrid(x_m, y_m)

    # perform transform; transformer.transform can accept ndarray
    lon2d, lat2d = transformer.transform(XX, YY)

    return lat2d, lon2d, ds['CMI'].values

# ---------------------------
# Frame processing
# ---------------------------

def process_peak_frame(frame_num, hour, minute):
    print(f"\n[{frame_num:02d}] Processing {hour:02d}:{minute:02d} UTC...")
    ds = get_peak_milton_data(PEAK_YEAR, PEAK_MONTH, PEAK_DAY, hour, minute)
    if ds is None:
        return None

    try:
        lat_grid, lon_grid, cmi = convert_goes_coordinates_pyproj(ds)
        temp_c = cmi - 273.15
        print(f"  ✅ Projection successful: lat {lat_grid.shape}, lon {lon_grid.shape}, data {cmi.shape}")
    except Exception as e:
        print(f"  ❌ Projection failed: {e}")
        return None

    # Mask valid data
    valid_mask = ~np.isnan(temp_c) & np.isfinite(lon_grid) & np.isfinite(lat_grid)
    if not np.any(valid_mask):
        print("  ❌ No valid data after masking")
        return None

    lon_flat = lon_grid[valid_mask]
    lat_flat = lat_grid[valid_mask]
    temp_flat = temp_c[valid_mask]

    print(f"  📊 Total valid points: {len(lon_flat):,}")

    # Filter to Gulf region where Milton was
    gulf_mask = (lon_flat > -95) & (lon_flat < -75) & (lat_flat > 20) & (lat_flat < 32)

    if np.any(gulf_mask):
        lon_flat = lon_flat[gulf_mask]
        lat_flat = lat_flat[gulf_mask]
        temp_flat = temp_flat[gulf_mask]
        print(f"  🌊 Gulf region points: {len(lon_flat):,}")
    else:
        print("  ⚠️ No points in Gulf, using all points")

    if len(lon_flat) == 0:
        print("  ❌ No data after filtering")
        return None

    # Downsample for D3 / export
    D3_MAX_POINTS = 2000
    if len(lon_flat) > D3_MAX_POINTS:
        step = max(1, len(lon_flat) // D3_MAX_POINTS)
        idx = np.arange(0, len(lon_flat), step)
        lon_flat = lon_flat[idx]
        lat_flat = lat_flat[idx]
        temp_flat = temp_flat[idx]
        print(f"  ⬇️ Downsampled to: {len(lon_flat):,} points")

    min_temp = np.nanmin(temp_flat)

    # Category mapping (based on min brightness temp C)
    if min_temp < -82:
        category, wind_mph = 5, 160
    elif min_temp < -77:
        category, wind_mph = 4, 140
    elif min_temp < -72:
        category, wind_mph = 3, 125
    elif min_temp < -67:
        category, wind_mph = 2, 105
    elif min_temp < -62:
        category, wind_mph = 1, 90
    else:
        category, wind_mph = 0, 75

    # Estimate center from coldest pixels
    cold_mask = temp_flat < -50
    if np.any(cold_mask):
        center_lon = float(np.mean(lon_flat[cold_mask]))
        center_lat = float(np.mean(lat_flat[cold_mask]))
    else:
        hours_from_12 = (hour + minute / 60.0) - 12.0
        center_lon = -86.0 + hours_from_12 * 0.67
        center_lat = 24.5 + hours_from_12 * 0.17

    df = pd.DataFrame({
        'lon': lon_flat,
        'lat': lat_flat,
        'temp_c': temp_flat,
        'temp_k': temp_flat + 273.15
    })

    df['frame'] = frame_num
    df['hour'] = hour
    df['minute'] = minute
    df['time_str'] = f"{hour:02d}:{minute:02d}"
    df['category'] = int(category)
    df['wind_mph'] = int(wind_mph)
    df['center_lon'] = center_lon
    df['center_lat'] = center_lat
    df['min_temp_c'] = float(min_temp)

    filename = f"milton_cat5_{frame_num:02d}.csv"
    filepath = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(filepath, index=False)

    print(f"  ✅ Saved to {filename}")
    print(f"  🌡️ Min temp: {min_temp:.1f}°C")
    print(f"  🌀 Category: {category}")
    print(f"  💨 Wind: ~{wind_mph} mph")
    print(f"  📍 Center: {center_lon:.3f}°W, {center_lat:.3f}°N")

    return df

# ---------------------------
# Main: test one frame then process all frames
# ---------------------------


In [11]:
print("=" * 70)
print("TESTING PROJECTION WITH ONE FRAME...")
print("=" * 70)

test_df = process_peak_frame(0, 12, 0)

if test_df is not None:
    print(f"\n✅ TEST SUCCESSFUL! Processing all frames...")
    print(f"   Sample data shape: {test_df.shape}")
    print(f"   Columns: {test_df.columns.tolist()}")

    # Build frames list
    frames = []
    frame_num = 0
    for hour in range(PEAK_START_HOUR, PEAK_END_HOUR + 1):
        for minute in (0, 30):
            # skip half-hour beyond end hour
            if hour == PEAK_END_HOUR and minute == 30:
                continue
            frames.append({'frame_num': frame_num, 'hour': hour, 'minute': minute})
            frame_num += 1

    print(f"\nProcessing {len(frames)} frames...")

    processed = []
    for f in frames:
        if f['frame_num'] == 0:
            processed.append(test_df)
            continue
        df = process_peak_frame(f['frame_num'], f['hour'], f['minute'])
        if df is not None:
            processed.append(df)
        time.sleep(0.5)

    print("\n" + "=" * 70)
    print("PROCESSING COMPLETE!")
    print("=" * 70)

    if processed:
        # Save timeline
        summary = {
            "hurricane": "Milton",
            "year": PEAK_YEAR,
            "peak_category": 5,
            "total_frames": len(processed),
            "interval_minutes": INTERVAL_MINUTES,
            "frames": []
        }

        for f in frames:
            filename = f"milton_cat5_{f['frame_num']:02d}.csv"
            filepath = os.path.join(OUTPUT_DIR, filename)
            if os.path.exists(filepath):
                try:
                    row = pd.read_csv(filepath, nrows=1)
                    summary["frames"].append({
                        "frame": f['frame_num'],
                        "hour": f['hour'],
                        "minute": f['minute'],
                        "filename": filename,
                        "time_str": f"{f['hour']:02d}:{f['minute']:02d}",
                        "category": int(row['category'].iloc[0]),
                        "wind_mph": int(row['wind_mph'].iloc[0]),
                        "center_lon": float(row['center_lon'].iloc[0]),
                        "center_lat": float(row['center_lat'].iloc[0])
                    })
                except Exception:
                    summary["frames"].append({
                        "frame": f['frame_num'],
                        "hour": f['hour'],
                        "minute": f['minute'],
                        "filename": filename,
                        "time_str": f"{f['hour']:02d}:{f['minute']:02d}",
                        "category": 5,
                        "wind_mph": 160
                    })
        timeline_path = os.path.join(OUTPUT_DIR, "timeline.json")
        with open(timeline_path, 'w') as fh:
            json.dump(summary, fh, indent=2)

        print(f"\n✅ SUCCESS: Created {len(processed)} frames")
        print(f"📁 Output: {OUTPUT_DIR}/")
        print(f"📋 Timeline: {timeline_path}")
    else:
        print("\n❌ No frames processed")
else:
    print("\n❌ Test failed. No output produced.")


TESTING PROJECTION WITH ONE FRAME...

[00] Processing 12:00 UTC...
  📡 Target: 12:00Z  ->  Actual: 2024-10-08T12:01:17Z  (OR_ABI-L2-CMIPC-M6C13_G16_s20242821201173_e20242821203558_c20242821204045.nc)
  ✅ Projection successful: lat (1500, 2500), lon (1500, 2500), data (1500, 2500)
  📊 Total valid points: 3,702,838
  🌊 Gulf region points: 528,522
  ⬇️ Downsampled to: 2,002 points
  ✅ Saved to milton_cat5_00.csv
  🌡️ Min temp: -81.7°C
  🌀 Category: 4
  💨 Wind: ~140 mph
  📍 Center: -85.194°W, 23.448°N

✅ TEST SUCCESSFUL! Processing all frames...
   Sample data shape: (2002, 13)
   Columns: ['lon', 'lat', 'temp_c', 'temp_k', 'frame', 'hour', 'minute', 'time_str', 'category', 'wind_mph', 'center_lon', 'center_lat', 'min_temp_c']

Processing 13 frames...

[01] Processing 12:30 UTC...
  📡 Target: 12:30Z  ->  Actual: 2024-10-08T12:31:17Z  (OR_ABI-L2-CMIPC-M6C13_G16_s20242821231173_e20242821233558_c20242821234058.nc)
  ✅ Projection successful: lat (1500, 2500), lon (1500, 2500), data (1500, 2500